In [1]:
import numpy as np
import pandas as pd
from IPython.display import clear_output

import sys
sys.path.append("..")
from utils import utils, model_utils, find_all_colnames
import fine_tuning

### Sentiment

In [2]:
checkpoints_path = "E:/TFM_CCIL/colab_checkpoints/"
table = fine_tuning.get_fine_tune_scores("sentiment", checkpoints_path)
table = utils.order_table(table, experiment="acl")
find_all_colnames.run()

Updated utils/all_colnames.tsv


In [4]:
for path, col in zip(["../data_exploration/acl/tables/basic_stats_sentiment_mbert.xlsx",
                      "../data_exploration/acl/tables/sentiment_balance.xlsx",
                      "../fine_tuning/class_weights_acl.xlsx",
                      "../fine_tuning/class_weights_acl.xlsx",
                      "../data_exploration/acl/tables/sentiment_balance.xlsx"],
                     ["train_examples",
                      "Ratio",
                      "Negative",
                      "Positive",
                      "Balanced_total"]):
    aux_table = pd.read_excel(path)
    table = utils.merge_tables(table, aux_table, how="left", cols_table2=[col])

In [5]:
basic_stats_mbert = pd.read_excel("../data_exploration/acl/tables/basic_stats_sentiment_mbert.xlsx")
basic_stats_xlm_roberta = pd.read_excel("../data_exploration/acl/tables/basic_stats_sentiment_xlm-roberta.xlsx")

In [6]:
basic_stats_mbert["model_name"] = "bert-base-multilingual-cased"
basic_stats_xlm_roberta["model_name"] = "tf-xlm-roberta-base"
basic_stats = pd.concat([basic_stats_mbert, basic_stats_xlm_roberta])

In [7]:
table = pd.merge(table, basic_stats[["language", "model_name", "train_avg_tokens"]], how="left", 
                 left_on=["training_lang", "model_name"], right_on=["language", "model_name"]).drop("language", axis=1)

In [8]:
def get_real_train_size(x):
    return x["train_examples"] if x["use_class_weights"] else x["Balanced_total"]

table["real_train_size"] = table[["use_class_weights", "train_examples", "Balanced_total"]].apply(get_real_train_size, axis=1)

In [12]:
table = table.rename(columns={"Ratio": "pos_ratio", "Negative": "neg_class_weight", 
                              "Positive": "pos_class_weight", "Balanced_total": "balanced_train_examples"})

In [13]:
table.to_excel("training_summary_acl_sentiment.xlsx", index=False)